In [1]:
import stim
c=stim.Circuit('''
H 0
Cnot 0 1
M 0 1
''')
#hadarmard on 0th qubit, ... measure both qubits

In [8]:
c.compile_sampler().sample(4)

array([[False, False],
       [ True,  True],
       [False, False],
       [ True,  True]])

In [12]:
#retetition code:
def rep_code(distance,rounds):
    circuit=stim.Circuit()
    qubits=range(2*distance+1)
    data=qubits[::2]
    measure=qubits[1::2]
    for m in measure:
        circuit.append_operation("CNOT",[m-1,m])
    for m in measure:
        circuit.append_operation("CNOT",[m+1,m])
    circuit.append_operation("MR",measure)
    return circuit*rounds

rep_code(distance=3,rounds=100)

stim.Circuit('''
    REPEAT 100 {
        CX 0 1 2 3 4 5 2 1 4 3 6 5
        MR 1 3 5
    }
''')

In [21]:
#CX 0 1 2 3 means CNOT 0 1
#               CNOT 2 3
qubits=range(2*3+1)
pairs1=qubits[:-1] #-1 is the 2nd from last
pairs2=qubits[1:]
pairs3=qubits[1:][::-1]
print(pairs1)
print(pairs2)
print(pairs3)

range(0, 6)
range(1, 7)
range(6, 0, -1)


In [22]:
def rep_code(distance,rounds):
    circuit=stim.Circuit()
    qubits=range(2*distance+1)
    data=qubits[::2]
    measure=qubits[1::2]
    pairs1=qubits[:-1] #-1 is the 2nd from last
    pairs2=qubits[1:][::-1]
    circuit.append_operation("CNOT",pairs1)
    circuit.append_operation("CNOT",pairs2)
    circuit.append_operation("MR",measure)
    return circuit*rounds

In [24]:
rep_code(distance=3,rounds=2).compile_sampler().sample(1)

array([[False, False, False, False, False, False]])

In [49]:
def shot(circuit):
    sample=circuit.compile_sampler().sample(1)[0]
    s=""
    for e in sample:
        if str(e)=="False":
            s=s+"_"
        else:
            s=s+"1"
    return s
shot(rep_code(distance=3,rounds=2,noise=0.01))

'______'

In [46]:
#noise is the probability for X error
def rep_code(distance,rounds,noise):
    circuit=stim.Circuit()
    qubits=range(2*distance+1)
    data=qubits[::2]
    measure=qubits[1::2]
    pairs1=qubits[:-1] #-1 is the 2nd from last
    pairs2=qubits[1:][::-1]
    circuit.append_operation("X_ERROR",data,noise)
    #circuit.append_operation("DEPOLARIZE",qubits,noise)
    circuit.append_operation("CNOT",pairs1)
    circuit.append_operation("CNOT",pairs2)
    circuit.append_operation("MR",measure)
    return circuit*rounds
shot(rep_code(distance=3,rounds=2,noise=0.01))

'000000'

In [28]:
import shutil
shutil.get_terminal_size().columns
#os.get_terminal_size() method in Python is used to query the size of a terminal. 
#This method returns the size of a terminal as a pair columns and lines. 
#Here, columns represents width of the terminal window in characters and lines 
#represents height of the terminal window in characters.



80

In [47]:
nc=shutil.get_terminal_size().columns

In [50]:
shot(rep_code(distance=nc,rounds=20,noise=0.01))

'________11_________________________11___________________________________________________11_________________________11___________________________________________________11_________________________11___________________________________________________11___11____________________11___________________________________________________11___11__11________________11_________________________11________________________11___11__11_______________1_1____11___________________11________________________11___11__11_______________1_1____11___________________11________________________11___11__11_______________1_1____11___________________11________________________11___11__11______________1__1____11___________________11________________________11___11__11______________1__1____11___________________11________________________11___11__11__11__________1__1_111_1___________________11___________11___________11___11__11__11__________1__1_111_1___________________11___________11___________11___1_1_11__11__________1__1_1

In [60]:
def rep_code(distance,rounds,noise):
    circuit=stim.Circuit()
    qubits=range(2*distance+1)
    data=qubits[::2]
    measure=qubits[1::2]
    pairs1=qubits[:-1] #-1 is the 2nd from last
    pairs2=qubits[1:][::-1]
    #circuit.append_operation("DEPOLARIZE2",pairs1,noise)
    
    circuit.append_operation("CNOT",pairs1)
    circuit.append_operation("DEPOLARIZE2",pairs1,noise)
    
    circuit.append_operation("CNOT",pairs2)
    circuit.append_operation("DEPOLARIZE2",pairs2,noise)
    
    circuit.append_operation("DEPOLARIZE1",qubits,noise)#single qubits depo.. error
    circuit.append_operation("MR",measure)
    
    for k in range(len(measure)):
        circuit.append_operation("DETECTOR",[stim.target_rec(-1-k),stim.target_rec(-1-distance-k)])
        
        
    #circuit.append_operation("DETECTOR",[stim.target_rec(-1),stim.target_rec(-1-distance)])
    #compare the last measurement with the measurement on the same qubits in the previous round.Because the 
    #number of measurements is the distance of the code
    
    full_circuit=stim.Circuit()
    full_circuit.append_operation("M",measure)
    full_circuit+=circuit*rounds
    return full_circuit
rep_code(distance=3,rounds=2,noise=0.01)

stim.Circuit('''
    M 1 3 5
    REPEAT 2 {
        CX 0 1 2 3 4 5
        DEPOLARIZE2(0.01) 0 1 2 3 4 5
        CX 6 5 4 3 2 1
        DEPOLARIZE2(0.01) 6 5 4 3 2 1
        DEPOLARIZE1(0.01) 0 1 2 3 4 5 6
        MR 1 3 5
        DETECTOR rec[-1] rec[-4]
        DETECTOR rec[-2] rec[-5]
        DETECTOR rec[-3] rec[-6]
    }
''')

In [61]:
def detect_shot(circuit):
    sample=circuit.compile_detector_sampler().sample(1)[0]
    s=""
    for e in sample:
        if str(e)=="False":
            s=s+"_"
        else:
            s=s+"1"
    return s
detect_shot(rep_code(distance=nc,rounds=20,noise=0.01))

'___________________________________________________1______________________________________1_____________________________1_________1______________________________________1_____________________________1____________11________________________________________________11___________1_______________________11____11________________11______________________________1___________________________________________________________________________________________________________________________________________11___________________________1_________________________________1____________________________________________1___________________11_____1_______1_______________________________________________________________11______1__________________________________________1_______________________11_______________________1______________11______________1__________________1____________________________1___1_________________11___________________1_______1_______________________________1___1______________________________